# **Modelo de clasificación de cáncer cervical en muestras de tejido celular utilizando técnicas de aprendizaje automático:** Monografía presentada​ para optar al título de ​Especialista en Analítica y Ciencia de Datos​

**Estudiantes:** <br>
Daniel Alberto López Sánchez, Ing Electronico. e-mail: daniel.lopez6@udea.edu.co <br>
July Andrea Muñoz Lopera, Ing Industrial. e-mail: july.munoz@udea.edu.co

**Asesora:** <br>
Maria Bernarda Salazar Sánchez, PhD. e-mail: bernarda.salazar@udea.edu.co

<br>
Universidad de Antioquia <br>
​Facultad de Ingeniería​ <br>
​​Especialización en Analítica y Ciencia de Datos​ <br>
​​Medellín, Antioquia, Colombia​ <br>
2023

# Objetivo
Este notebook tiene como objetivo extraer información relevante de las imágenes de cada una de las clases. A través de estos descriptores se puede identificar y capturar patrones y características específicas de las imágenes, las cuales pueden ser utilizadas para clasificar y agrupar las imágenes en diferentes categorías.

# Configuración de librerías, APIS y descarga de dataset

En esta sección, se presentan módulos clave que son fundamentales para el desarrollo del experimento. Específicamente, estos módulos se encuentran alojados en el repositorio de GitHub "MonografiaDataScience", el cual se descarga previamente mediante el comando ```git clone https://github.com/Alberto-San/MonografiaDataScience```. El comando anterior crea una carpeta en el espacio de colab con el nombre MonografiaDataScience, la cual contiene diversas funciones necesarias para la extracción de características.

In [ ]:
!git clone https://github.com/Alberto-San/MonografiaDataScience

fatal: destination path 'MonografiaDataScience' already exists and is not an empty directory.


## Base de datos Sipakmed
La Base de datos SIPaKMeD consiste en 4049 imágenes de células aisladas que han sido recortadas manualmente a partir de 966 imágenes de grupos de células en láminas de Papanicolaou. Estas imágenes fueron adquiridas a través de una cámara CCD adaptada a un microscopio óptico. Las imágenes de células se dividen en cinco categorías que contienen células normales, anormales y benignas.

Las 5 grandes categorías son:
* Koilocytotic
* Superficial-Intermediate
* Metaplastic
* Dyskeratotic
* Parabasal



Para descargar la base de datos, se necesitan seguir algunos pasos:

* Instalar la biblioteca kaggle en el entorno de Colab, mediante el comando ```pip install kaggle```.
* Descargar el archivo JSON de la API-key de Kaggle a través de la página web de Kaggle.
* Configurar las variables de entorno que requiere la biblioteca kaggle, mediante los comandos ```"os.environ['KAGGLE_USERNAME'] = '%%USERNAME%%'"``` y ```"os.environ['KAGGLE_KEY'] = '%%API_KEY%%'"```.
* Descargar la base de datos utilizando el comando ```!kaggle datasets download -d prahladmehandiratta/cervical-cancer-largest-dataset-sipakmed -p sipakmed --unzip```.


Para este caso en particular, y con el objetivo de reproducir el experimento fácilmente, se deja visible el API-key usado, y se crea el archivo JSON de manera manual, al igual que la configuración de las variables de entorno.

In [ ]:
import json

# Cargar archivo JSON en una cadena
api_token = '{"username":"daniellpez2","key":"cdfc04075fcd22835f527b5f5fa40af7"}'

# Convertir cadena en un objeto JSON
api_token = json.loads(api_token)

# Crear archivo temporal que contiene el contenido de la cadena
with open('kaggle.json', 'w') as file:
    json.dump(api_token, file)

# Establecer variables de entorno para la API de Kaggle
import os
os.environ['KAGGLE_USERNAME'] = api_token['username']
os.environ['KAGGLE_KEY'] = api_token['key']

Se instala la librería y se descarga la base de datos como se mencionó en los pasos anteriores.

In [ ]:
!pip install kaggle
!kaggle datasets download -d prahladmehandiratta/cervical-cancer-largest-dataset-sipakmed -p sipakmed --unzip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
100% 6.38G/6.40G [01:03<00:00, 117MB/s]
100% 6.40G/6.40G [01:03<00:00, 109MB/s]


# **Características**

Una vez que el conjunto de datos ha sido descargado, el siguiente paso es extraer las características del mismo. Para esto, se sigue el procedimiento descrito por *Plissiti et al* en su artículo **Sipakmed: A New Dataset for Feature and Image Based Classification of Normal and Pathological Cervical Cells in Pap Smear Images**, en el cual extraen las siguientes características para cada una de las dimensiones de color (R, G, B) de las imágenes:
* Intensidad promedio: Se define como la media de la imagen
* Suavidad: Se define como el promedio del valor absoluto de la imagen al aplicar un filtro laplaciano
* Uniformidad: Se define como las suma de las diferencias al cuadrado entre la imagen al aplicar un filtro guassiano, y la imagen al aplicar multiples veces el filtro gaussiano
* Tercer Momento: Se define como el tercer momento de la distribucion de la imagen como vector. 
* Entropia: Se define como la entropia de shannon al considerar la distribucion de frecuencias de la imagen como distribucion de probabilidad
* Desviacion estandar: Se define como la desviacion estandar de la imagen
* Mediana: Se define como la mediana de la imagen
* Contraste promedio: Se define como la media de la luminancia de la imagen


Para llevar a cabo la extracción de características mencionada anteriormente, se utiliza el módulo ```FeaturesAPI/Experiment_color``` del paquete ```MonografiaDataScience```. Específicamente, se utilizan las clases ```CervicalCancerContentReader```, ```CalculateImageClasicalFeatures``` y la interfaz de variables ```constants```.


In [ ]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Funcion auxiliar que permite accesar a los modulos de MonografiaDataScience
def cd_into(folder):
  sys.path.append(os.path.abspath(os.path.join(os.path.dirname("."), folder)))

# Acceso a los modulos de FeaturesAPI/Experiment_color
cd_into("MonografiaDataScience/FeaturesAPI/Experiment_color")

In [ ]:
# Importando librerias y funcionalidades requeridas
import cv2
from scipy.stats import moment
from CervicalCancerContentReader import *
from CalculateImageClasicalFeatures import *
from constants import *
import multiprocessing

# Funcion que permite obtener las componentes R,G, B de una imagen y permite escalarla a la resolucion objetivo:
# (220, 220)
def get_color_component(path):
    print("path: {}".format(path))
    image_bgr = cv2.imread(path)
    color_component = cv2.resize(image_bgr, size, interpolation=cv2.INTER_LINEAR)
    return color_component

# Funcion que permite obtener la informacion de los descriptores previamente mencionados por cada canal (R, G, B)
# Ademas de esto, recolecta la informacion de la clase a la que pertenece el descriptor y el path de la imagen
def get_descriptor(args):
    index, label, image_paths, image_component, hanlder = args
    return [label, image_paths[index]] + hanlder.calculate_features(image_component)

# Funcion que permite obtener la informacion de los descriptores previamente mencionados por cada canal (R, G, B)
# haciendo uso de logica multiproceso
def calculate_features_label(label, categories_content):
    hanlder = CalculateImageClasicalFeatures("bgr") # se define el objeto de la funcion que permite calcular las caracteristicas
    image_paths = categories_content[label][IMAGE_PATH_KEY] # Se obtiene una lista de los paths de las imagenes que pertenecen a la categoria a procesar.
    num_processes = multiprocessing.cpu_count() # Se reconoce el numero de nucleos de la cpu de servidor de colab
    pool = multiprocessing.Pool(num_processes) 
    print("Reading Images")
    im_list = pool.map(get_color_component, image_paths) # Se ejecuta la funcion sobre los diferentes path de manera concurrente
    function_args = [(index, label, image_paths, im_list[index], hanlder) for index in range(len(im_list))]
    print("Getting Descriptors")
    features = pool.map(get_descriptor, function_args) # Se ejecuta la funcion sobre los diferentes datos de manera concurrente
    return features

Se define el proceso de extracion de caracteristicas de la siguiente manera:

1. Se lee el contenido del dataset
2. Se lee la informacion de las diferentes categorias/clases (Koilocytotic, Superficial-Intermediate, Metaplastic, Dyskeratotic, Parabasal).
3. Por cada una de las categorias se recolecta la siguiente informacion
  * Clase a la que pertenece: nombre de la categoria que se analiza
  * Path de la imagen: ruta relativa del path de la imagen analizada
  * Feature 1, ..., Feature N: caracteristicas previamente mencionadas que se recolentan de cada una de las imagenes.


In [25]:
import pandas as pd

sub_folder = "CROPPED" # Subfolder de la base de datos
image_extension = "bmp" # Extension de las imagenes de la base de datos
data_extension = "dat" # Extension de los archivos de metadatos de la base de datos
dataset_output_path = "/content/sipakmed" # carpeta donde se encuentra la informacion (imagenes) de la base de datos
size = (220, 220) # Resolucion objetivo para estandarizar cada uno de los recortes de imagenes de las diferentes clases 

# Funcionalidad personalizada que se encarga de leer la informacion del dataset SipakMed.
# La logica se encuentra en el respositorio MonografiaDataScience
categories_content = CervicalCancerContentReader(
                        dataset_output_path,
                        image_extension,
                        data_extension,
                        sub_folder
                    ).read() 

# Lectura de las diferentes categorias de la base de datos:  Koilocytotic, Superficial-Intermediate, Metaplastic
# Dyskeratotic, Parabasal
labels = list(categories_content.keys())

accum_list = [] # placeholder para guardar la informacion de los descriptores y los datos
accum_labels = [] # placeholder que contiene el nombre de las cabeceras
flag_define_labels = False

for label in labels:
    [accum_list.append(feature) for feature in calculate_features_label(label, categories_content)]

    if not(flag_define_labels):
        accum_labels = ["feature_{}".format(index) for index in range(len(accum_list[0])-2)]
        flag_define_labels = True
    
# Se define el nombre de las columnas
columns = [
    "class",
    "image_path"
    ] + accum_labels

# Se crea un dataframe con la informacion de los descriptores y datos
df = pd.DataFrame(accum_list, columns=columns)

# Se materializa el dataframe como archivo csv en colab
df.to_csv("color_statistics.csv", index=False)

Se ilustran a continuación las estadísticas de cada una de las columnas del archivo CSV.

In [26]:
df.describe()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21
count,4049.000000,4049.000000,4049.000000,4049.000000,4049.000000,4049.000000,4049.000000,4049.000000,4049.000000,4049.000000,...,4049.000000,4049.000000,4049.000000,4049.000000,4049.000000,4049.000000,4049.000000,4049.000000,4049.000000,4049.000000
mean,156.325574,0.013050,0.050097,4937.435138,6.274991,28.444748,154.467152,132.117775,0.014407,0.067247,...,31.419329,129.097555,135.178617,0.011919,0.055613,1595.657946,6.463401,31.622616,133.915658,135.793304
std,47.200665,0.006448,0.069130,36217.317387,0.730330,13.414944,50.284998,40.053187,0.006888,0.064934,...,11.709597,42.639157,41.219059,0.005760,0.051680,43740.354941,0.582921,11.665083,47.090754,37.045545
min,32.180145,0.001120,0.001113,-223977.273661,0.447899,3.332016,30.000000,38.393285,0.004412,0.000841,...,4.458147,38.000000,35.002831,0.001759,0.001292,-244531.850043,1.100130,5.755065,29.000000,47.520124
25%,122.668864,0.007893,0.009755,-5855.160978,5.899003,18.524319,118.000000,101.389215,0.008953,0.017568,...,22.781303,96.000000,104.870496,0.007378,0.018657,-14884.273243,6.153852,23.180675,101.000000,106.340971
50%,161.878223,0.011377,0.027184,-367.480824,6.346107,26.092095,160.000000,131.496260,0.012616,0.048946,...,29.973123,128.000000,135.945909,0.010153,0.040533,-1319.954314,6.564285,31.009116,135.000000,133.866074
75%,189.791550,0.017283,0.061226,8078.139421,6.783356,35.564597,190.000000,161.425103,0.019040,0.097459,...,38.831991,160.000000,163.001364,0.015499,0.075188,13361.744084,6.887963,39.073903,164.000000,164.516818
max,254.171756,0.034033,0.803293,366650.108983,7.621974,84.488579,255.000000,237.302521,0.051348,0.463266,...,83.300989,255.000000,253.127541,0.039157,0.409219,484126.083913,7.561561,90.110651,255.000000,232.263574


Se explorará más acerca de la distribución de los datos en el siguiente notebook titulado "Análisis de Distribución de Datos".

De igual manera, se descarga el archivo CSV y se guarda de manera **manual** en el repositorio MonografiaDataScience en la subcarpeta tmp (esto debido a que Colab no tiene permiso para realizar push directos al repositorio), con el fin de guardar las caracteristicas extraidas, y usarlas en el siguiente notebook.